In [ ]:
%%capture
import os
import pandas as pd
import numpy as np
from dj_notebook import activate
from pathlib import Path

env_file = os.environ["INTECOMM_ENV"]
documents_folder = Path(os.environ["INTECOMM_DOCUMENTS_FOLDER"])
plus = activate(dotenv_file=env_file)
report_folder = Path(documents_folder)


In [ ]:
from statsmodels.genmod.generalized_estimating_equations import GEE
from statsmodels.genmod.families import Binomial
from statsmodels.stats.proportion import proportion_confint


In [ ]:
path = documents_folder / 'df_bp.csv'
df_bp = pd.read_csv(path)
df_bp



In [ ]:
path = documents_folder / 'df_glucose_gee.csv'
df_glucose_gee = pd.read_csv(path)
df_glucose_gee


In [ ]:
df_gee = pd.merge(df_bp, df_glucose_gee, on=["subject_identifier","assignment", "time"], how="outer")

In [ ]:
def is_controlled(s):
    if pd.notna(s["bp_controlled"]) and pd.notna(s["glucose_controlled"]):
        if s["bp_controlled"] is True and s["glucose_controlled"] is True:
            return True
        elif s["bp_controlled"] is False and s["glucose_controlled"] is False:
            return False
        else:
            return True
    elif pd.notna(s["bp_controlled"]) and pd.isna(s["glucose_controlled"]):
        return s["bp_controlled"]
    elif pd.isna(s["bp_controlled"]) and pd.notna(s["glucose_controlled"]):
        return s["glucose_controlled"]
    else:
        return np.nan


df_gee["controlled"] = df_gee.apply(is_controlled, axis=1)
df_gee.drop(columns={"bp_controlled", "glucose_controlled"}, inplace=True)
df_gee

In [ ]:
df = df_gee.copy()

# Define the dependent variable and independent variables
dependent_var = 'controlled'
independent_vars = ['assignment', 'time']

# Convert categorical variables to dummy variables
df = pd.get_dummies(df, columns=['assignment', 'time'], drop_first=True)

# Update the list of independent variables after creating dummy variables
independent_vars = [col for col in df.columns if col not in ['controlled', 'subject_identifier']]

# Define the model
model = GEE(df[dependent_var], df[independent_vars], groups=df['subject_identifier'], family=Binomial())

# Fit the model
result = model.fit()
print(result.summary())

In [ ]:

df = df_gee.copy()

# Calculate the crude risk difference
risk_a = df[df['assignment'] == 'a']['controlled'].mean()
risk_b = df[df['assignment'] == 'b']['controlled'].mean()
crude_risk_difference = risk_b - risk_a

# Calculate the confidence interval for the crude risk difference
n_a = df[df['assignment'] == 'a'].shape[0]
n_b = df[df['assignment'] == 'b'].shape[0]
ci_low_a, ci_upp_a = proportion_confint(count=df[df['assignment'] == 'a']['controlled'].sum(), nobs=n_a, alpha=0.05, method='normal')
ci_low_b, ci_upp_b = proportion_confint(count=df[df['assignment'] == 'b']['controlled'].sum(), nobs=n_b, alpha=0.05, method='normal')

# Calculate the confidence interval for the risk difference
ci_low_diff = (risk_b - ci_upp_a) - (risk_a - ci_upp_b)
ci_upp_diff = (risk_b + ci_low_a) - (risk_a + ci_low_b)

print(f"Crude Risk Difference: {crude_risk_difference}")
print(f"95% Confidence Interval for Crude Risk Difference: ({ci_low_diff}, {ci_upp_diff})")